In [1]:
import pandas as pd
#from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit

## Model

In [2]:
def clicksPerAdDay(clicks):
    return pd.DataFrame({'clicksPerShows': clicks.groupby(['ad_id', 'day'])['clicked'].mean()
                        }).reset_index()

In [3]:
def join(series):
    return ' '.join( map(str, series) )

In [4]:
def model_prediction_day(clicksPerAdDay_train, test):
    by='clicksPerShows'
    
    meanDay = pd.DataFrame({by: clicksPerAdDay_train.groupby(['day'])[by].mean() }).reset_index()
    
    prediction = pd.merge(test, clicksPerAdDay_train, how='left', on=['ad_id', 'day'])
    
    prediction = prediction[prediction[by].notnull()].append(
                            pd.merge(prediction[prediction[by].isnull()].drop(by, axis=1), meanDay, how='left', on=['day']) )
                           
    prediction = prediction.fillna(clicksPerAdDay_train[by].mean())
    prediction = prediction.sort_values(by=['display_id', by], ascending=False)
                           
    return pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).reset_index()

In [5]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

## Validation

In [6]:
data = pd.read_csv("../generated/clicks_docs_train.csv")
data.count()

display_id     87141731
document_id    87141731
day            87141731
ad_id          87141731
clicked        87141731
dtype: int64

In [7]:
displays = pd.read_csv("../download/events.csv", usecols=['display_id'])
displays.count()

display_id    23120126
dtype: int64

In [8]:
#kf = KFold(n_splits=5)
#train, test = kf.split(displays).next()
ss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=0)
train_index, test_index = ss.split(displays).next()

print len(train_index), len(test_index)

17340094 5780032


In [9]:
clicksPerAdDay_train = clicksPerAdDay( pd.merge(data, displays.iloc[train_index], on=['display_id']) )
clicks_test_sample = pd.merge(data, displays.iloc[test_index], on=['display_id'])
del displays
validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str})

In [10]:
print model_evaluation(validation, model_prediction_day(clicksPerAdDay_train, clicks_test_sample))

0.643245181675


## Submission

In [7]:
clicksPerAdDay_train = pd.read_csv("../generated/clicks_shows_PerAdDay.csv", usecols=['ad_id', 'day', 'clicksPerShows'])

In [8]:
clicks_test = pd.read_csv("../generated/clicks_docs_test.csv")

In [17]:
prediction = model_prediction_day(clicksPerAdDay_train, clicks_test)

In [ ]:
prediction.head()

In [19]:
prediction.to_csv("../submissions/solution_3.csv", index=False)